In [1]:
import os

In [2]:
%pwd

'C:\\Users\\hp\\Documents\\Projects\\Python\\SkinCancer\\SkinCancer-Classification\\research'

In [3]:
#os.chdir("../")

In [4]:
%pwd

'C:\\Users\\hp\\Documents\\Projects\\Python\\SkinCancer\\SkinCancer-Classification'

In [49]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    

In [50]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [51]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path= CONFIG_FILE_PATH,
        params_file_path= PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])
        
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir =os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir),Path(config.tensorboard_root_log_dir)])
        
        prepare_callback_config = PrepareCallbacksConfig(root_dir=Path(config.root_dir),
                                                         tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
                                                         checkpoint_model_filepath=Path(config.checkpoint_model_filepath))
        
        return prepare_callback_config
        

In [52]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [57]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
        
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir, f"tb_logs_at_{timestamp}")
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        
        return tf.keras.callbacks.ModelCheckpoint(filepath=self.config.checkpoint_model_filepath, save_best_only=True)
    
    
    
    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callbacks, self._create_ckpt_callbacks]


In [58]:
try: 
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2024-10-15 10:26:35,959: INFO: common: Reading the yaml file from config\config.yaml]
[2024-10-15 10:26:35,959: INFO: common: Reading the yaml file from params.yaml]
[2024-10-15 10:26:35,974: INFO: common: Directory created at artifacts]
[2024-10-15 10:26:35,974: INFO: common: Directory created at artifacts\prepare_callbacks\checkpoint_dir]
[2024-10-15 10:26:35,974: INFO: common: Directory created at artifacts\prepare_callbacks\tensorboard_log_dir]


In [47]:
import time
timestamp = time.strftime("%Y-%m-%d-%H:%M:%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2024-10-15-10:20:40'

In [ ]:
def _create_ckpt_callbacks(self):
        # Convertir el Path a str para usar endswith
        checkpoint_filepath = str(self.config.checkpoint_model_filepath)
        if checkpoint_filepath.endswith(".h5"):
            checkpoint_filepath = checkpoint_filepath.replace(".h5", ".keras")
        
        return tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_best_only=True)